## Esse notebook é um exemplo de uso de regras de associação usando Orange Lib

Para execução do código, será necessário instalar as seguintes bibliotecas:

pip3 install numpy

pip3 install scipy

pip3 install orange3

In [1]:
from orangecontrib.associate.fpgrowth import *  
import Orange

In [2]:
# Create simple transactions data
T = [[1, 3, 4],
    [2, 3, 5],
    [1, 2, 3, 5],
    [2, 5]]

**Min support** = Qual a frequência minima de uma regra?

In [6]:
min_support = 2
itemsets = frequent_itemsets(T, min_support)

print("Itemsets found : \n\n")
for item,freq in list(itemsets):
    print('Itemset: ' +repr(item) + ' has frequency of ' + repr(freq))
    
rules = [(P, Q, supp, conf)
    for P, Q, supp, conf in association_rules(dict(frequent_itemsets(T, min_support)), .01)]

print("\n\nAssociation rules found : \n\n")
for ante, cons, supp, conf in rules[:5]:
    print(', '.join(str(i) for i in ante), '-->',
        next(iter(cons)),
        '(supp: {}, conf: {})'.format(supp, conf))


Itemsets found : 


Itemset: frozenset({1}) has frequency of 2
Itemset: frozenset({2}) has frequency of 3
Itemset: frozenset({3}) has frequency of 3
Itemset: frozenset({1, 3}) has frequency of 2
Itemset: frozenset({2, 3}) has frequency of 2
Itemset: frozenset({5}) has frequency of 3
Itemset: frozenset({2, 5}) has frequency of 3
Itemset: frozenset({3, 5}) has frequency of 2
Itemset: frozenset({2, 3, 5}) has frequency of 2


Association rules found : 


3, 5 --> 2 (supp: 2, conf: 1.0)
2, 5 --> 3 (supp: 2, conf: 0.6666666666666666)
5 --> 2 (supp: 2, conf: 0.6666666666666666)
2, 3 --> 5 (supp: 2, conf: 1.0)
3 --> 2 (supp: 2, conf: 0.6666666666666666)


## Zoo Dataset
Vamos agora usar esse conhecimento em um dataset real.

In [4]:
data = Orange.data.Table('zoo')
print('data = \n'+ repr(data[0:10]))
X, mapping = OneHot.encode(data, include_class=True)
print('\nX (one hode encoding)= \n'+ repr(X) + ' of shape '+ repr(X.shape))
# print('\nMapping = \n' + repr(mapping.items()))
itemsets = dict(frequent_itemsets(X, .4))

data = 
[[1, 0, 0, 1, 0, ... | mammal] {aardvark},
 [1, 0, 0, 1, 0, ... | mammal] {antelope},
 [0, 0, 1, 0, 0, ... | fish] {bass},
 [1, 0, 0, 1, 0, ... | mammal] {bear},
 [1, 0, 0, 1, 0, ... | mammal] {boar},
 ...
]

X (one hode encoding)= 
array([[False,  True,  True, ..., False,  True, False],
       [False,  True,  True, ..., False,  True, False],
       [ True, False,  True, ..., False, False, False],
       ..., 
       [False,  True,  True, ..., False,  True, False],
       [ True, False,  True, ...,  True, False, False],
       [ True, False, False, ..., False, False, False]], dtype=bool) of shape (101, 43)


### E é claro, precisamos analisar as regras geradas! :)

In [5]:
class_items = {item
    for item, var, _ in OneHot.decode(mapping, data, mapping)
    if var is data.domain.class_var}

rules = [(P, Q, supp, conf)
    for P, Q, supp, conf in association_rules(itemsets, .8)
    if len(Q) == 1 and Q & class_items]

names = {item: '{}={}'.format(var.name, val)
    for item, var, val in OneHot.decode(mapping, data, mapping)}

for ante, cons, supp, conf in rules[:5]:
    print(', '.join(names[i] for i in ante), '-->',
        names[next(iter(cons))],
        '(supp: {}, conf: {})'.format(supp, conf))


backbone=1, feathers=0, breathes=1, venomous=0, milk=1 --> type=mammal (supp: 41, conf: 1.0)
backbone=1, feathers=0, breathes=1, milk=1 --> type=mammal (supp: 41, conf: 1.0)
backbone=1, breathes=1, venomous=0, milk=1 --> type=mammal (supp: 41, conf: 1.0)
feathers=0, breathes=1, venomous=0, milk=1 --> type=mammal (supp: 41, conf: 1.0)
backbone=1, feathers=0, breathes=1, venomous=0 --> type=mammal (supp: 41, conf: 0.8723404255319149)


## Desafio
Implemente outas métricas de avaliação, assim como Lift.